# Dev Jokes ASCII Art AI Generator

This script generates ASCII art of developer jokes using Azure OpenAI Dall·e and the `ascii_magic` library.

In [1]:
import os
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
import json

from ascii_magic import AsciiArt

# Data preparation

We need to remove some quotes and other strange characters before generating the pictures.

In [2]:
import pandas as pd

df = pd.read_excel("developer_jokes_3.xlsx")
df

,Joke
0,// TODO: Actually do this.
1,A clean codebase is a sign of a postponed dead...
2,AI can code in every language. Understanding t...
3,AI didn’t replace me. It just gave me more wei...
4,AI explained transformers using actual Transfo...
...,...
628,We deploy on Fridays. We also like chaos.
629,We use canary deploys. The canary’s not lookin...
630,We’ll fix that in the next sprint.
631,"Yes, I need to see the error. No, a photo of y..."


In [3]:
df['Joke'][3]

'AI didn’t replace me. It just gave me more weird bugs to debug.'

In [4]:
# find non ascii characters on dataframe
non_ascii_chars = set(char for joke in df['Joke'] for char in str(joke) if ord(char) > 127)
print(sorted(non_ascii_chars))

['ö', '—', '‘', '’', '“', '”', '…']


Not a big deal, we can live with that.

In [5]:
endpoint = os.getenv(
    "AZURE_OPENAI_ENDPOINT", "https://cog-7qxr3pddyatvm.openai.azure.com/"
)
api_version = os.getenv("OPENAI_API_VERSION", "2024-04-01-preview")
dalle_deployment = os.getenv("DALL_E_DEPLOYMENT_NAME", "dall-e-3")
gpt_deployment = os.getenv("GPT_DEPLOYMENT_NAME", "gpt-4.1-nano")

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
)

In [11]:
import time
from IPython.display import Image, display, clear_output
import requests
from urllib.parse import urlparse


def generate_image_from_joke(joke:str)->str:
    messages = [
        {
            "role": "system",
            "content": "You are a helpful designer that ideates images based on jokes. The images are abstract ideas of the joke. They will be converted to ASCII art, so don't overdo it or add any text, as it won't be visible once converted.",
        },
        {
            "role": "user",
            "content": "Joke: %s\nIdea: " % joke,
        },
    ]

    # Generate the description of the image based on the joke
    print(f"Generating description for joke: {joke}")
    completion = client.chat.completions.create(
        model=gpt_deployment,
        messages=messages,
        max_tokens=800,
        temperature=1,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
    )

    print(f"Description: {completion.choices[0].message.content}")

    result = client.images.generate(
        model=dalle_deployment,
        prompt=completion.choices[0].message.content,
        n=1,
        style="vivid",
        quality="standard",
    )
    image_url = json.loads(result.model_dump_json())["data"][0]["url"]
    # download the image
    if not image_url:
        raise ValueError("No image URL returned from the model.")

    return image_url

try:
    for index,row in df[:11].iterrows():
        idx=str(index).zfill(4)
        image_filename=f"images/image_{idx}.png"

        if os.path.exists(image_filename):
            print(f"Image already exists: {image_filename}")            
        else:
            retry_count = 0
            while retry_count < 3:
                try:
                    image_url = generate_image_from_joke(row["Joke"])
                    break
                except Exception as e:
                    print(f"Error generating image: {e}")
                    retry_count += 1
                    time.sleep(5 ** retry_count + 1)  # Exponential backoff
                    if retry_count >= 3:
                        print("Failed to generate image after 3 attempts.")
                        continue

            # Download the image
            print(f"Downloading image from URL: {image_url}")
            response = requests.get(image_url)
            if response.status_code != 200:
                raise ValueError(
                    f"Failed to download image, status code: {response.status_code}"
                )

            with open(image_filename, "wb") as f:
                f.write(response.content)

        df.at[index, "image"] = image_filename
        ascii_art = AsciiArt.from_image(image_filename)
        fname= f"art/ascii_art_{idx}.html"
        ascii_art.to_html_file(f"../docs/{fname}")        
        df.at[index, "ascii_art"] = fname
finally:    
    df.to_json("../docs/developer_jokes.json", orient="records", indent=2)

Image already exists: images/image_0000.png
Image already exists: images/image_0001.png
Image already exists: images/image_0002.png
Image already exists: images/image_0003.png
Image already exists: images/image_0004.png
Image already exists: images/image_0005.png
Image already exists: images/image_0006.png
Image already exists: images/image_0007.png
Image already exists: images/image_0008.png
Image already exists: images/image_0009.png
Image already exists: images/image_0010.png


In [7]:
df.head(10)


,Joke,image,ascii_art
0,// TODO: Actually do this.,images/image_0000.png,art/ascii_art_0000.html
1,A clean codebase is a sign of a postponed dead...,images/image_0001.png,art/ascii_art_0001.html
2,AI can code in every language. Understanding t...,images/image_0002.png,art/ascii_art_0002.html
3,AI didn’t replace me. It just gave me more wei...,images/image_0003.png,art/ascii_art_0003.html
4,AI explained transformers using actual Transfo...,images/image_0004.png,art/ascii_art_0004.html
5,AI generated 100 lines of code and a new impos...,images/image_0005.png,art/ascii_art_0005.html
6,AI generated a README longer than the app itself.,images/image_0006.png,art/ascii_art_0006.html
7,AI helped build the app. It also helped crash it.,images/image_0007.png,art/ascii_art_0007.html
8,AI named my functions after feelings. It's not...,images/image_0008.png,art/ascii_art_0008.html
9,AI promised productivity. Delivered prompt anx...,images/image_0009.png,art/ascii_art_0009.html
